In [4]:
!pip install pyECLAT

In [5]:
# Import necessary modules
import numpy as np
import pandas as pd

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from pyECLAT import ECLAT

In [ ]:
# load the dataset
df = pd.read_csv("/content/Walmart_market_data.csv", header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,nut,lemon,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,iced tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,lemon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,iced tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Checking dataframe info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7501 entries, 0 to 7500
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       7501 non-null   object
 1   1       5747 non-null   object
 2   2       4389 non-null   object
 3   3       3345 non-null   object
 4   4       2529 non-null   object
 5   5       1864 non-null   object
 6   6       1369 non-null   object
 7   7       981 non-null    object
 8   8       654 non-null    object
 9   9       395 non-null    object
 10  10      256 non-null    object
 11  11      154 non-null    object
 12  12      87 non-null     object
 13  13      47 non-null     object
 14  14      25 non-null     object
 15  15      8 non-null      object
 16  16      4 non-null      object
 17  17      4 non-null      object
 18  18      3 non-null      object
 19  19      1 non-null      object
dtypes: object(20)
memory usage: 1.1+ MB


In [ ]:
# Cleaning and converting the data
data = []
for row in range(df.shape[0]):
  data.append([i for i in df.values[row, :] if type(i) != float])

data[: 5]

[['shrimp',
  'nut',
  'lemon',
  'vegetables mix',
  'green grapes',
  'whole weat flour',
  'yams',
  'cottage cheese',
  'energy drink',
  'tomato juice',
  'low fat yogurt',
  'iced tea',
  'honey',
  'salad',
  'mineral water',
  'salmon',
  'antioxydant juice',
  'frozen smoothie',
  'spinach',
  'olive oil'],
 ['burgers', 'meatballs', 'eggs'],
 ['chutney'],
 ['turkey', 'lemon'],
 ['mineral water', 'milk', 'energy bar', 'whole wheat rice', 'iced tea']]

In [ ]:
# Transforming the dataset
te = TransactionEncoder()
t_data = te.fit_transform(data)

t_df = pd.DataFrame(t_data, columns=te.columns_)
t_df.head()

,asparagus,antioxydant juice,asparagus,babies food,bacon,barbecue sauce,black tea,blueberries,body spray,bramble,brownies,bug spray,burger sauce,burgers,butter,cake,candy bars,carrots,cauliflower,cereals,champagne,chicken,chili,chocolate,chocolate bread,chutney,cider,clothes accessories,cookies,cooking oil,corn,cottage cheese,cream,dessert wine,eggplant,eggs,energy bar,energy drink,escalope,extra dark chocolate,...,oil,olive oil,pancakes,parmesan cheese,pasta,pepper,pet food,pickles,protein bar,red wine,rice,salad,salmon,salt,sandwich,shallot,shampoo,shrimp,soda,soup,spaghetti,sparkling water,spinach,strawberries,strong cheese,tea,tomato juice,tomato sauce,tomatoes,toothpaste,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,True,False,False,True,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False


In [ ]:
# Create apriori model and fit the data on that
freq = apriori(t_df, min_support=0.01, use_colnames=True)
freq

,support,itemsets
0,0.010799,(barbecue sauce)
1,0.014265,(black tea)
2,0.011465,(body spray)
3,0.033729,(brownies)
4,0.087188,(burgers)
...,...,...
252,0.011065,"(milk, mineral water, ground beef)"
253,0.017064,"(spaghetti, mineral water, ground beef)"
254,0.015731,"(milk, spaghetti, mineral water)"
255,0.010265,"(spaghetti, mineral water, olive oil)"


In [ ]:
# Create a new column to get the length of each pair
itemssets_len = []
for i in freq['itemsets']:
  itemssets_len.append(len(i))

freq['len'] = itemssets_len
freq.head()

,support,itemsets,len
0,0.010799,(barbecue sauce),1
1,0.014265,(black tea),1
2,0.011465,(body spray),1
3,0.033729,(brownies),1
4,0.087188,(burgers),1


In [ ]:
# Sorting the dataframe by support column
sorted_freq = freq.sort_values(by=['support'], ascending=False, ignore_index=True)
sorted_freq.head()

,support,itemsets,len
0,0.238368,(mineral water),1
1,0.179709,(eggs),1
2,0.174110,(spaghetti),1
3,0.170911,(french fries),1
4,0.163845,(chocolate),1


In [ ]:
# Checking total number of pairs
sorted_freq['len'].value_counts()

2    165
1     75
3     17
Name: len, dtype: int64

In [ ]:
# Get top 10 most valid pairs
count = 0
top_items = []

for i in sorted_freq.values:
  if count >= 10:
    break
  elif i[2] > 1:
    top_items.append(i[1])
    count += 1
  
top_items = pd.Series(top_items)
top_items

0            (spaghetti, mineral water)
1            (mineral water, chocolate)
2                 (eggs, mineral water)
3                 (milk, mineral water)
4          (mineral water, ground beef)
5              (spaghetti, ground beef)
6                (spaghetti, chocolate)
7                     (eggs, spaghetti)
8                  (french fries, eggs)
9    (mineral water, frozen vegetables)
dtype: object

In [ ]:
# Get top 5 for different len
for k in range(1, 4):
  count = 0
  top5_items = []

  for i in sorted_freq.values:
    if count >= 10:
      break
    elif i[2] == k:
      top5_items.append(i[1])
      count += 1

  top5_items = pd.Series(top5_items)
  print(f"Top 5 values for {k} pairs")
  print("-" * 50)
  print(top5_items)
  print("\n\n\n")

Top 5 values for 1 pairs
--------------------------------------------------
0        (mineral water)
1                 (eggs)
2            (spaghetti)
3         (french fries)
4            (chocolate)
5             (iced tea)
6                 (milk)
7          (ground beef)
8    (frozen vegetables)
9             (pancakes)
dtype: object




Top 5 values for 2 pairs
--------------------------------------------------
0            (spaghetti, mineral water)
1            (mineral water, chocolate)
2                 (eggs, mineral water)
3                 (milk, mineral water)
4          (mineral water, ground beef)
5              (spaghetti, ground beef)
6                (spaghetti, chocolate)
7                     (eggs, spaghetti)
8                  (french fries, eggs)
9    (mineral water, frozen vegetables)
dtype: object




Top 5 values for 3 pairs
--------------------------------------------------
0          (spaghetti, mineral water, ground beef)
1            (chocolate, spaghetti,

In [ ]:
# Create model using eclate
instance = ECLAT(data=df, verbose=True)

100%|██████████| 120/120 [00:00<00:00, 2239.68it/s]


In [ ]:
# Fit the data on the instance
index, suppport = instance.fit(min_support=0.05)

Combination 1 by 1


25it [00:00, 67.02it/s]


Combination 2 by 2


300it [00:02, 125.68it/s]


Combination 3 by 3


2300it [00:18, 121.58it/s]


In [ ]:
# Check support values
suppport

{'burgers': 0.0871883748833489,
 'cake': 0.08105585921877083,
 'chicken': 0.05999200106652446,
 'chocolate': 0.1638448206905746,
 'cookies': 0.08038928142914278,
 'cooking oil': 0.0510598586855086,
 'eggs': 0.17970937208372217,
 'eggs & mineral water': 0.05092654312758299,
 'escalope': 0.0793227569657379,
 'french fries': 0.1709105452606319,
 'frozen smoothie': 0.06332489001466471,
 'frozen vegetables': 0.09532062391681109,
 'grated cheese': 0.0523930142647647,
 'ground beef': 0.09825356619117451,
 'iced tea': 0.13211571790427942,
 'low fat yogurt': 0.07652313024930009,
 'milk': 0.12958272230369283,
 'mineral water': 0.23836821757099053,
 'mineral water & chocolate': 0.05265964538061592,
 'mineral water & spaghetti': 0.05972536995067324,
 'olive oil': 0.0658578856152513,
 'pancakes': 0.09505399280095987,
 'shrimp': 0.07145713904812692,
 'soup': 0.05052659645380616,
 'spaghetti': 0.17411011865084655,
 'tomatoes': 0.06839088121583789,
 'turkey': 0.06252499666711105,
 'whole wheat rice': 